In [38]:
from PIL import Image, ImageDraw, ImageFont
import random

class Draw_Table():
    def __init__(self) -> None:
        # Possible column names
        self.column_names = ['Description', 'Quantity', 'Tax %', 'Netto', 'Brutto']
        # List of possible items in the table
        self.items = [
            'Electricity',
            'Water',
            'Gas',
            'Printer',
            'Maintenance fee',
            'Desktop monitor',
            'Designing cost',
            'Office chair',
            'Cell phone',
            'Laptop',
            'Network access point',
            'Network switch',
            'Professional beamer',
            'Smart white board',
            'Thermostat',
            'Office plants',
            'Professional software',
            'Printer ink',
            'Cleaning material',
            'Keyboard',
            'Network cable',
            'Security camera',
            'Carpet',
            'Washing machine',
            'Kitchen sink',
            'Garbage can'
        ]

    def __call__(self, drawer, bbox, currency):
        self.draw = ImageDraw.Draw(drawer)
        self.bbox = bbox                            # Bounding box of the table [x1, y1, x2, y2]
        self.currency = currency                    # CUrrency to use in the table
        self.font = ImageFont.truetype("arial.ttf", 30)
        self.num_items = random.randint(3, 10)      # How many rows does the table have (number of items)
        self.num_cols = random.randint(2, 5)        # How many columns does the table have
        self.increment = 50                         # Height of a row in pixels
        self.margin = 12                            # Distance between the table lines and text
        self.labels = []                            # Store the labels of the words

        self.draw_horizontal_lines()
        self.draw_vertical_lines()
        self.draw_header()
        self.draw_content()

        return self.labels

    def text_box(self, draw, coords, text, label, font, anchor='lt'):
        # Draw the text on the canvas
        draw.text(coords, text, fill='black', font=font, anchor=anchor)
        # Get the bounding box of the text
        box = draw.textbbox(coords, text, font=font)
        # Get the bounding box of each word

        # Add the new label to the labels list
        self.labels.append(
            {
                'box': box,
                'text': text,
                'label': label
            }
        )
        return 

    def get_x_positions(self):
        # Width of the table x2 - x1
        full_width = self.bbox[2] - self.bbox[0]
        col_width = int(full_width / self.num_cols)
        # X coordinate of the beginning of a column
        col_positions = []
        # The table has an offset, add this offset to the columns too
        pos = self.bbox[0]

        for _ in range(self.num_cols):
            col_positions.append(pos)
            pos += col_width
        col_positions.append(pos)

        return col_positions

    def draw_horizontal_lines(self):
        left_x = self.bbox[0]
        left_y = self.bbox[1]
        right_x = self.bbox[2]
        right_y = self.bbox[1]

        for _ in range(self.num_items):
            left_y += self.increment
            right_y += self.increment
            self.draw.line([(left_x, left_y), (right_x, right_y)], fill ="black", width = 2)

    def draw_vertical_lines(self):
        top_y = self.bbox[1] + self.increment
        bot_y = self.bbox[1] + (self.increment * self.num_items)

        positions = self.get_x_positions()

        self.draw.line([(self.bbox[0], top_y), (self.bbox[0], bot_y)], fill ="black", width = 2)
        for pos_x in positions:
            self.draw.line([(pos_x, top_y), (pos_x, bot_y)], fill ="black", width = 2)

    def draw_brutto(self, bruttos):
        for row in range(self.num_items - 1): 
            y = self.bbox[1] + row * self.increment + self.increment + self.margin
            # Draw the currency
            box = self.text_box((self.bbox[2] - self.margin, y), self.currency, self.font, 'rt')
            #self.draw.text((self.bbox[2] - self.margin, y), self.currency, fill='black', font=self.font, anchor='rt')
            # Get the width of the currency
            #curr_width, curr_height = self.draw.textsize(self.currency, font=self.font)
            # Draw the amount
            self.draw.text((self.bbox[2] - curr_width - 20, y), str(bruttos[row]), fill='black', font=self.font, anchor='rt')

    def draw_header(self):
        positions = self.get_x_positions()
        
        for col in range(self.num_cols - 1):
            self.draw.text((positions[col] + 10, self.bbox[1]), self.column_names[col], fill='black', font=self.font, anchor='lt', stroke_width=1)
        self.draw.text((positions[-1] - 10, self.bbox[1]), self.column_names[-1], fill='black', font=self.font, anchor='rt', stroke_width=1)

    def draw_content(self):
        descriptions = random.sample(self.items, self.num_items-1)
        quantities = random.sample(range(20), self.num_items-1)
        taxes = random.sample(range(5, 20), self.num_items-1)
        bruttos = random.sample(range(5000), self.num_items-1)
        nettos = [a-a*b*0.01 for a,b in zip(bruttos,taxes)]
        
        data = [descriptions, quantities, taxes, nettos]

        positions = self.get_x_positions()

        for row in range(self.num_items - 1):
            for col in range(self.num_cols - 1):
                self.draw.text((positions[col] + self.margin, self.bbox[1]+row*self.increment+self.increment+self.margin), str(data[col][row]), fill='black', font=self.font, anchor='lt')

        self.draw_brutto(bruttos)
        self.draw_sum_field(str(sum(bruttos)))

    def draw_sum_field(self, summa):
        x = self.bbox[2] - self.margin
        self.draw.text((x, self.bbox[1] + self.increment*self.num_items + self.margin), self.currency, fill='black', font=self.font, anchor='rt')
        text_width, text_height = self.draw.textsize(self.currency, font=self.font)
        self.draw.text((x - text_width - 8, self.bbox[1] + self.increment*self.num_items + self.margin), summa, fill='black', font=self.font, anchor='rt')
        text_width2, text_height = self.draw.textsize(summa, font=self.font)
        self.draw.text((x - text_width - text_width2 - 8, self.bbox[1] + self.increment*self.num_items + self.margin), 'Sum: ', fill='black', font=self.font, anchor='rt', stroke_width=1)

In [24]:
class LayoutManager():
    def __init__(self) -> None:
        pass

    def __call__(self):
        layout = {
            'R_field': [100, 100, 600, 500],    # Recipient
            'S_field': [900, 350, 1538, 850],   # Supplier
            'L_field': [1000, 50, 1538, 200],   # Logo
            'T_field': [100, 1000, 1538, 1600], # Table
            'I_field': [100, 2000, 1538, 2200], # Information
            'Q_field': [550, 550, 800, 800],    # QR code
            'X_field': [100, 1650, 1538, 1900]  # Text field
        }

        return layout

In [25]:
class Draw_customer_field():
    def __init__(self) -> None:
        pass

    def __call__(self):
        pass

In [39]:
from fakedata import FakeData

class InvoiceGenerator():
    def __init__(self, canvas, num_documents = 1, det = True) -> None:
        self.canvas = canvas
        self.drawer = ImageDraw.Draw(self.canvas)
        self.font = ImageFont.truetype("arial.ttf", 30)
        self.num_documents = num_documents
        self.det = det        

    def __call__(self):
        # 1. Generate the layout
        layout_manager = LayoutManager()
        self.layout = layout_manager()
        # self.visualize_layout()

        # 2. Generate the fake data
        fake_data_generator = FakeData()
        fake_data = fake_data_generator(1, True)

        # 3. Draw the fake data on the image
        table_drawer = Draw_Table()
        for document in fake_data:
            table_drawer(self.canvas, self.layout['T_field'], document['I_Currency'])
            self.canvas.show()

    def visualize_layout(self):
        for key in self.layout:
            self.drawer.rectangle(self.layout[key], outline='black')
            self.drawer.text(self.layout[key], key, font=self.font, fill='black')
        self.canvas.show()

blank_dir = r'C:\Users\Habram\Documents\Datasets\fake-invoices\blank.tif'
blank = Image.open(blank_dir)

invoice_generator = InvoiceGenerator(blank, 1, True)
invoice_generator()

C:\Users\Habram\AppData\Local\Temp\ipykernel_16416\2451828174.py:101: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  curr_width, curr_height = self.draw.textsize(self.currency, font=self.font)
C:\Users\Habram\AppData\Local\Temp\ipykernel_16416\2451828174.py:101: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  curr_width, curr_height = self.draw.textsize(self.currency, font=self.font)
C:\Users\Habram\AppData\Local\Temp\ipykernel_16416\2451828174.py:101: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  curr_width, curr_height = self.draw.textsize(self.currency, font=self.font)
C:\Users\Habram\AppData\Local\Temp\ipykernel_16416\2451828174.py:101: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or te

In [58]:
blank_dir = r'C:\Users\Habram\Documents\Datasets\fake-invoices\blank.tif'
blank = Image.open(blank_dir)
draw = ImageDraw.Draw(blank)
font = ImageFont.truetype("arial.ttf", 30)

def text_box(draw, coords, text, font, anchor='lt'):
    draw.text(coords, text, fill='black', font=font, anchor=anchor)
    text_box = draw.textbbox(coords, text, font=font)
    
    words = text.split()
    for word in words:
        word_box = draw.textlength(word)
        print(word)

bbox = text_box(draw, [100, 100], 'Habram Istvan kiraly', font)
#draw.rectangle(bbox, outline='red')
blank.show()

Habram
Istvan
kiraly
